In [8]:
import pandas as pd
from rdflib import Graph, URIRef
from typing import Optional

def execute_sparql_and_get_dataframe(
    kg: Graph, 
    sparql_query: str, 
    print_results: bool = True
) -> Optional[pd.DataFrame]:
    """
    Führt eine SPARQL-Abfrage auf einem rdflib-Graphen aus und gibt die 
    Ergebnisse als Pandas DataFrame zurück.

    Args:
        kg: Der rdflib Graph (Knowledge Graph).
        sparql_query: Der SPARQL-Query-String (muss eine SELECT-Abfrage sein).
        print_results: Bool, um die Ergebnisse im Klartext auszugeben.

    Returns:
        Ein Pandas DataFrame mit den Ergebnissen oder None, falls pandas nicht 
        verfügbar oder die Abfrage ungültig ist.
    """
    
    print("--- STARTE ABFRAGE ---")
    
    try:
        # Abfrage ausführen
        results = kg.query(sparql_query)
        
        # Prüfen, ob Ergebnisse vorhanden sind und ob es sich um eine SELECT-Abfrage handelt
        if not hasattr(results, 'vars'):
            print("❌ Fehler: Die Abfrage hat keine Variablen (vermutlich keine SELECT-Abfrage).")
            return None
        
        # Ausgabe der Ergebnisse
        if print_results:
            print(f"### SPARQL-Abfrage-Ergebnisse: {len(results)} Zeilen gefunden ###")
            
            # Die Namen der SELECT-Variablen
            var_names = [str(v) for v in results.vars]
            print(f"Variablen: {', '.join(var_names)}")
            print("-" * (len(var_names) * 10))

            # Beispielhafte Ausgabe der ersten paar Zeilen
            for i, row in enumerate(results):
                if i >= 5000 and len(results) > 5000:
                    print(f"... und {len(results) - i} weitere Zeilen.")
                    break
                
                # Formatierung der Zeile: Variablenname: Wert
                row_output = []
                for var_name, value in zip(results.vars, row):
                    # Nur den letzten Teil der URI anzeigen (QName-ähnlich)
                    display_value = str(value).split('/')[-1]
                    row_output.append(f"{var_name.toPython()}: {display_value}")
                
                print(" | ".join(row_output))
            print("-" * (len(var_names) * 10))


        # Konvertierung in Pandas DataFrame
        try:
            # Erstellt einen DataFrame direkt aus den Ergebnissen
            df = pd.DataFrame(results, columns=list(results.vars))
            
            # URIRef-Objekte in Strings konvertieren, um sie leichter lesbar zu machen
            for col in df.columns:
                if isinstance(df[col].iloc[0], (URIRef)):
                    df[col] = df[col].apply(lambda x: str(x))
            
            print("\n✅ Ergebnisse als Pandas DataFrame erfolgreich erstellt.")
            return df
        
        except ImportError:
            print("❌ Fehler: 'pandas' ist nicht installiert. Installieren Sie es mit: !pip install pandas")
            return None
            
    except Exception as e:
        print(f"❌ Ein Fehler ist bei der SPARQL-Abfrage aufgetreten: {e}")
        return None

In [ ]:
#HW-Variablen-Pfade finden
kg = Graph()
kg.parse(r"D:\MA_Python_Agent\MSRGuard\KGs\ParamDiag_Agent_filled2.ttl", format='turtle')

q="""
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX ag:  <http://www.semanticweb.org/AgentProgramParams/>
PREFIX dp:  <http://www.semanticweb.org/AgentProgramParams/dp_>
PREFIX op:  <http://www.semanticweb.org/AgentProgramParams/op_>
PREFIX cl: <http://www.semanticweb.org/AgentProgramParams/cl_>

SELECT DISTINCT *
WHERE {
ag:Program_AxisControl_MultipleInputSensors op:hasInputVariable ?vin.
?vin op:isMappedToVariable ?vmap.
#op:hasOutputVariable ?vout.
?vprog op:isMappedToProgram ag:Program_AxisControl_MultipleInputSensors.
 }
"""
execute_sparql_and_get_dataframe(kg, q)



--- STARTE ABFRAGE ---
### SPARQL-Abfrage-Ergebnisse: 2850 Zeilen gefunden ###
Variablen: vprog, vmap, vin
------------------------------
?vprog: Var_HRL_CB_AS_HorizontalMoveSensors_HRL_CB_HorizontalMove | ?vmap: Var_HRL_CB_AS_HorizontalMoveSensors_LightBarrierSensor_01 | ?vin: Var_AxisControl_MultipleInputSensors_DI01
?vprog: Var_HRL_CB_AS_HorizontalMoveSensors_HRL_CB_HorizontalMove | ?vmap: Var_HRL_RGB_AS_HorizontalMoveSensors_TouchSensor01 | ?vin: Var_AxisControl_MultipleInputSensors_DI01
?vprog: Var_HRL_CB_AS_HorizontalMoveSensors_HRL_CB_HorizontalMove | ?vmap: Var_HRL_RGB_InitialStateDrive_localId:10000000005 | ?vin: Var_AxisControl_MultipleInputSensors_DI01
?vprog: Var_HRL_CB_AS_HorizontalMoveSensors_HRL_CB_HorizontalMove | ?vmap: Var_HRL_RGB_InitialStateDrive_localId:20000000004 | ?vin: Var_AxisControl_MultipleInputSensors_DI01
?vprog: Var_HRL_CB_AS_HorizontalMoveSensors_HRL_CB_HorizontalMove | ?vmap: Var_HRL_RGB_InitialStateDrive_localId:30000000004 | ?vin: Var_AxisControl_Mult

,vprog,vmap,vin
0,http://www.semanticweb.org/AgentProgramParams/...,http://www.semanticweb.org/AgentProgramParams/...,http://www.semanticweb.org/AgentProgramParams/...
1,http://www.semanticweb.org/AgentProgramParams/...,http://www.semanticweb.org/AgentProgramParams/...,http://www.semanticweb.org/AgentProgramParams/...
2,http://www.semanticweb.org/AgentProgramParams/...,http://www.semanticweb.org/AgentProgramParams/...,http://www.semanticweb.org/AgentProgramParams/...
3,http://www.semanticweb.org/AgentProgramParams/...,http://www.semanticweb.org/AgentProgramParams/...,http://www.semanticweb.org/AgentProgramParams/...
4,http://www.semanticweb.org/AgentProgramParams/...,http://www.semanticweb.org/AgentProgramParams/...,http://www.semanticweb.org/AgentProgramParams/...
...,...,...,...
2845,http://www.semanticweb.org/AgentProgramParams/...,http://www.semanticweb.org/AgentProgramParams/...,http://www.semanticweb.org/AgentProgramParams/...
2846,http://www.semanticweb.org/AgentProgramParams/...,http://www.semanticweb.org/AgentProgramParams/...,http://www.semanticweb.org/AgentProgramParams/...
2847,http://www.semanticweb.org/AgentProgramParams/...,http://www.semanticweb.org/AgentProgramParams/...,http://www.semanticweb.org/AgentProgramParams/...
2848,http://www.semanticweb.org/AgentProgramParams/...,http://www.semanticweb.org/AgentProgramParams/...,http://www.semanticweb.org/AgentProgramParams/...


In [13]:
q="""
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX ag:  <http://www.semanticweb.org/AgentProgramParams/>
PREFIX dp:  <http://www.semanticweb.org/AgentProgramParams/dp_>
PREFIX op:  <http://www.semanticweb.org/AgentProgramParams/op_>
PREFIX cl: <http://www.semanticweb.org/AgentProgramParams/cl_>

SELECT DISTINCT *
WHERE {
?prog op:isSubProgramOf ag:Program_HRL_SkillSet.
?var op:isMappedToProgram ?prog.
?prog op:hasInputVariable ?ivar.
?ivar op:isMappedToVariable ?mvar.
 }
"""
execute_sparql_and_get_dataframe(kg, q)

--- STARTE ABFRAGE ---
### SPARQL-Abfrage-Ergebnisse: 104 Zeilen gefunden ###
Variablen: var, mvar, ivar, prog
----------------------------------------
?var: Var_HRL_SkillSet_HRL_ControlOfOutputs | ?mvar: Var_HRL_OperatingModes_A1 | ?ivar: Var_HRL_ControlOfOutputs_A1 | ?prog: Program_HRL_ControlOfOutputs
?var: Var_HRL_SkillSet_HRL_ControlOfOutputs | ?mvar: Var_HRL_OperatingModes_A2 | ?ivar: Var_HRL_ControlOfOutputs_A2 | ?prog: Program_HRL_ControlOfOutputs
?var: Var_HRL_SkillSet_HRL_ControlOfOutputs | ?mvar: Var_HRL_OperatingModes_A6 | ?ivar: Var_HRL_ControlOfOutputs_A6 | ?prog: Program_HRL_ControlOfOutputs
?var: Var_HRL_SkillSet_HRL_ControlOfOutputs | ?mvar: Var_HRL_SkillSet_HRL_CB_backwards | ?ivar: Var_HRL_ControlOfOutputs_CB_backwards | ?prog: Program_HRL_ControlOfOutputs
?var: Var_HRL_SkillSet_HRL_ControlOfOutputs | ?mvar: Var_HRL_SkillSet_HRL_CB_forwards | ?ivar: Var_HRL_ControlOfOutputs_CB_forwards | ?prog: Program_HRL_ControlOfOutputs
?var: Var_HRL_SkillSet_HRL_ControlOfOutputs 

,var,mvar,ivar,prog
0,http://www.semanticweb.org/AgentProgramParams/...,http://www.semanticweb.org/AgentProgramParams/...,http://www.semanticweb.org/AgentProgramParams/...,http://www.semanticweb.org/AgentProgramParams/...
1,http://www.semanticweb.org/AgentProgramParams/...,http://www.semanticweb.org/AgentProgramParams/...,http://www.semanticweb.org/AgentProgramParams/...,http://www.semanticweb.org/AgentProgramParams/...
2,http://www.semanticweb.org/AgentProgramParams/...,http://www.semanticweb.org/AgentProgramParams/...,http://www.semanticweb.org/AgentProgramParams/...,http://www.semanticweb.org/AgentProgramParams/...
3,http://www.semanticweb.org/AgentProgramParams/...,http://www.semanticweb.org/AgentProgramParams/...,http://www.semanticweb.org/AgentProgramParams/...,http://www.semanticweb.org/AgentProgramParams/...
4,http://www.semanticweb.org/AgentProgramParams/...,http://www.semanticweb.org/AgentProgramParams/...,http://www.semanticweb.org/AgentProgramParams/...,http://www.semanticweb.org/AgentProgramParams/...
...,...,...,...,...
99,http://www.semanticweb.org/AgentProgramParams/...,http://www.semanticweb.org/AgentProgramParams/...,http://www.semanticweb.org/AgentProgramParams/...,http://www.semanticweb.org/AgentProgramParams/...
100,http://www.semanticweb.org/AgentProgramParams/...,http://www.semanticweb.org/AgentProgramParams/...,http://www.semanticweb.org/AgentProgramParams/...,http://www.semanticweb.org/AgentProgramParams/...
101,http://www.semanticweb.org/AgentProgramParams/...,http://www.semanticweb.org/AgentProgramParams/...,http://www.semanticweb.org/AgentProgramParams/...,http://www.semanticweb.org/AgentProgramParams/...
102,http://www.semanticweb.org/AgentProgramParams/...,http://www.semanticweb.org/AgentProgramParams/...,http://www.semanticweb.org/AgentProgramParams/...,http://www.semanticweb.org/AgentProgramParams/...


In [ ]:
import time

def MBS_SkillSet(StartButton, GVL.diNotAus_Kanal_A, GVL.diNotAus_Kanal_B, ResetButton, ConfirmationButton, MBS_MR01_InitialStateDrive.ResetMethodCallInitDriveMR01_Backwards, MBS_MR01_InitialStateDrive.ResetMethodCallInitDriveMR01_Forwards, MBS_VSG_InitialStateDrive.ResetMethodCallInitDriveVSG, MBS_DmPD_InitialStateDrive.ResetMethodCallRotateCCW, MBS_DmPD_InitialStateDrive.ResetMethodCallRotateCW, MBS_MR02_InitialStateDrive.ResetMethodCallCuttingProcess, MBS_OperatingModes.F4, MBS_OperatingModes.F1, MBS_OperatingModes.A2, MBS_OperatingModes.A1, MBS_OperatingModes.A6, MBS_OperatingModes.Start, CB_MoveForwards, DmPD_Compressor_HorizontalMove, DmPD_Valve_HorizontalMove, DmPD_RotateCWformVSGtoMR02, DmPD_RotateCWromVSGtoCB, DmPD_RotateCWfromMR02toCB, DmPD_RotateCCWformCBtoMR02, DmPD_RotateCCWformCBtoVSG, DmPD_RotateCCWformMR02toVSG, MR01_HardeningProcess, MR01_MoveOvenslideIn, MR01_MoveOvenslideOut, MR01_Compressor_SecuringProcess, MR01_Valve_OvenDoor, MR02_CuttingProcess, VSG_MoveHorizontalForwards, VSG_MoveHorizontalBackwards, VSG_Valve_VerticalMove, VSG_Valve_SuctionProcessToDmPD, VSG_Valve_SuctionProcessToMR01, VSG_Compressor_VerticalMove, VSG_Compressor_SuctionProcessToDmPD, VSG_Compressor_SuctionProcessToMR01, Init_DmPD_RotateCCW, Init_DmPD_RotateCW, Init_MR01_HorizontalForwards, Init_MR01_HorizontalBackwards, Init_MR02_CuttingProcess, Init_VSG_HorizontalForwards, Init_VSG_HorizontalBackwards, MethodCall_InitStateDrive, InputVar_InitStateDrive, GVL_MBS.MBS_Ref_Taster_Ofenschieber_innen, GVL_MBS.MBS_Ref_Taster_Ofenschieber_aussen, GVL_MBS.MBS_Ref_Taster_Sauger_Pos_DK, TIME_TSR, GVL_MBS.MBS_Ref_Taster_DK_Pos_Foerderband, GVL_MBS.MBS_Ref_Taster_DK_Pos_Sauger, InitTime_Cutting, PowerOnDelay_Compressor, PowerOnDelay_Valve_VerticalMove, GVL_MBS.MBS_Ref_Taster_Sauger_Pos_Ofen, PowerOnDelay_Valve_SuctionProcess, GVL_MBS.MBS_Ref_Taster_DK_Pos_Saege, PowerOnDelay_Valve_HorizontalMove, GVL_MBS.MBS_LS_Ende_Foerderband, PowerOffDelay_ConveyorBelt):
    """Auto-generated from PLCopen XML (vereinfachte Semantik)."""

    def AND_1(V_10000000002, V_10000000003):
        result = GVL.diNotAus_Kanal_A and GVL.diNotAus_Kanal_B
        return result
    V_10000000004 = AND_1(GVL.diNotAus_Kanal_A, GVL.diNotAus_Kanal_B)

    def AND_2(V_10000000007, V_10000000008, V_10000000009, V_10000000010, V_10000000011, V_10000000012):
        result = MBS_MR01_InitialStateDrive.ResetMethodCallInitDriveMR01_Backwards and MBS_MR01_InitialStateDrive.ResetMethodCallInitDriveMR01_Forwards and MBS_VSG_InitialStateDrive.ResetMethodCallInitDriveVSG and MBS_DmPD_InitialStateDrive.ResetMethodCallRotateCCW and MBS_DmPD_InitialStateDrive.ResetMethodCallRotateCW and MBS_MR02_InitialStateDrive.ResetMethodCallCuttingProcess
        return result
    V_10000000013 = AND_2(MBS_MR01_InitialStateDrive.ResetMethodCallInitDriveMR01_Backwards, MBS_MR01_InitialStateDrive.ResetMethodCallInitDriveMR01_Forwards, MBS_VSG_InitialStateDrive.ResetMethodCallInitDriveVSG, MBS_DmPD_InitialStateDrive.ResetMethodCallRotateCCW, MBS_DmPD_InitialStateDrive.ResetMethodCallRotateCW, MBS_MR02_InitialStateDrive.ResetMethodCallCuttingProcess)

    def EXECUTE(V_10000000013):
        """EXECUTE block – original ST-Code:
MethodCall_InitStateDrive := FALSE;
InputVar_InitStateDrive := FALSE;"""
        result = V_10000000013
        return result

    def MBS_OperatingModes(V_10000000001, V_10000000004, V_10000000005, V_10000000006, V_10000000014):
        result = V_10000000014
        return result
    V_10000000015 = MBS_OperatingModes(StartButton, V_10000000004, ResetButton, ConfirmationButton, V_10000000014)

    def MBS_ControlOfOutputs(V_20000000000, V_20000000001, V_20000000002, V_20000000003, V_20000000004, V_20000000005, V_20000000006, V_20000000007, V_20000000008, V_20000000009, V_20000000010, V_20000000011, V_20000000012, V_20000000013, V_20000000014, V_20000000015, V_20000000016, V_20000000017, V_20000000018, V_20000000019, V_20000000020, V_20000000021, V_20000000022, V_20000000023, V_20000000024, V_20000000025, V_20000000026, V_20000000027, V_20000000028, V_20000000029, V_20000000030, V_20000000031, V_20000000032, V_20000000033, V_20000000034, V_20000000035):
        result = Init_VSG_HorizontalBackwards
        return result
    V_20000000036 = MBS_ControlOfOutputs(MBS_OperatingModes.F4, MBS_OperatingModes.F1, MBS_OperatingModes.A2, MBS_OperatingModes.A1, MBS_OperatingModes.A6, MBS_OperatingModes.Start, CB_MoveForwards, DmPD_Compressor_HorizontalMove, DmPD_Valve_HorizontalMove, DmPD_RotateCWformVSGtoMR02, DmPD_RotateCWromVSGtoCB, DmPD_RotateCWfromMR02toCB, DmPD_RotateCCWformCBtoMR02, DmPD_RotateCCWformCBtoVSG, DmPD_RotateCCWformMR02toVSG, MR01_HardeningProcess, MR01_MoveOvenslideIn, MR01_MoveOvenslideOut, MR01_Compressor_SecuringProcess, MR01_Valve_OvenDoor, MR02_CuttingProcess, VSG_MoveHorizontalForwards, VSG_MoveHorizontalBackwards, VSG_Valve_VerticalMove, VSG_Valve_SuctionProcessToDmPD, VSG_Valve_SuctionProcessToMR01, VSG_Compressor_VerticalMove, VSG_Compressor_SuctionProcessToDmPD, VSG_Compressor_SuctionProcessToMR01, Init_DmPD_RotateCCW, Init_DmPD_RotateCW, Init_MR01_HorizontalForwards, Init_MR01_HorizontalBackwards, Init_MR02_CuttingProcess, Init_VSG_HorizontalForwards, Init_VSG_HorizontalBackwards)

    def AND_3(V_30000000001, V_30000000002):
        result = GVL.diNotAus_Kanal_A and GVL.diNotAus_Kanal_B
        return result
    V_30000000003 = AND_3(GVL.diNotAus_Kanal_A, GVL.diNotAus_Kanal_B)

    def MBS_MR01_InitialStateDrive(V_30000000000, V_30000000003, V_30000000004, V_30000000005, V_30000000006, V_30000000007):
        result = MBS_OperatingModes.A6
        return result
    V_30000000008 = MBS_MR01_InitialStateDrive(MethodCall_InitStateDrive, V_30000000003, InputVar_InitStateDrive, GVL_MBS.MBS_Ref_Taster_Ofenschieber_innen, GVL_MBS.MBS_Ref_Taster_Ofenschieber_aussen, MBS_OperatingModes.A6)

    def AND_4(V_40000000001, V_40000000002):
        result = GVL.diNotAus_Kanal_A and GVL.diNotAus_Kanal_B
        return result
    V_40000000003 = AND_4(GVL.diNotAus_Kanal_A, GVL.diNotAus_Kanal_B)

    def MBS_VSG_InitialStateDrive(V_40000000000, V_40000000003, V_40000000004, V_40000000005, V_40000000006, V_40000000007):
        result = MBS_OperatingModes.A6
        return result
    V_40000000008 = MBS_VSG_InitialStateDrive(MethodCall_InitStateDrive, V_40000000003, InputVar_InitStateDrive, GVL_MBS.MBS_Ref_Taster_Sauger_Pos_DK, TIME_TSR, MBS_OperatingModes.A6)

    def AND_5(V_50000000001, V_50000000002):
        result = GVL.diNotAus_Kanal_A and GVL.diNotAus_Kanal_B
        return result
    V_50000000003 = AND_5(GVL.diNotAus_Kanal_A, GVL.diNotAus_Kanal_B)

    def MBS_DmPD_InitialStateDrive(V_50000000000, V_50000000003, V_50000000004, V_50000000005, V_50000000006, V_50000000007, V_50000000008):
        result = MBS_OperatingModes.A6
        return result
    V_50000000009 = MBS_DmPD_InitialStateDrive(MethodCall_InitStateDrive, V_50000000003, InputVar_InitStateDrive, GVL_MBS.MBS_Ref_Taster_DK_Pos_Foerderband, GVL_MBS.MBS_Ref_Taster_DK_Pos_Foerderband, GVL_MBS.MBS_Ref_Taster_DK_Pos_Sauger, MBS_OperatingModes.A6)

    def AND_6(V_60000000001, V_60000000002):
        result = GVL.diNotAus_Kanal_A and GVL.diNotAus_Kanal_B
        return result
    V_60000000003 = AND_6(GVL.diNotAus_Kanal_A, GVL.diNotAus_Kanal_B)

    def MBS_MR02_InitialStateDrive(V_60000000000, V_60000000003, V_60000000004, V_60000000005, V_60000000006):
        result = MBS_OperatingModes.A6
        return result
    V_60000000007 = MBS_MR02_InitialStateDrive(MethodCall_InitStateDrive, V_60000000003, InputVar_InitStateDrive, InitTime_Cutting, MBS_OperatingModes.A6)

    def AND_7(V_70000000000, V_70000000001):
        result = GVL.diNotAus_Kanal_A and GVL.diNotAus_Kanal_B
        return result
    V_70000000002 = AND_7(GVL.diNotAus_Kanal_A, GVL.diNotAus_Kanal_B)

    def MBS_MR01_AS_HorizontalMoveSensors_1(V_70000000002, V_70000000003, V_70000000004, V_70000000005, V_70000000006):
        result = MBS_OperatingModes.F1
        return result
    V_70000000007 = MBS_MR01_AS_HorizontalMoveSensors_1(V_70000000002, GVL_MBS.MBS_Ref_Taster_Ofenschieber_aussen, GVL_MBS.MBS_Ref_Taster_Ofenschieber_innen, TIME_TSR, MBS_OperatingModes.F1)

    def AND_8(V_80000000000, V_80000000001):
        result = GVL.diNotAus_Kanal_A and GVL.diNotAus_Kanal_B
        return result
    V_80000000002 = AND_8(GVL.diNotAus_Kanal_A, GVL.diNotAus_Kanal_B)

    def MBS_MR01_AS_HorizontalMoveSensors_2(V_80000000002, V_80000000003, V_80000000004, V_80000000005, V_80000000006):
        result = MBS_OperatingModes.F1
        return result
    V_80000000007 = MBS_MR01_AS_HorizontalMoveSensors_2(V_80000000002, GVL_MBS.MBS_Ref_Taster_Ofenschieber_innen, GVL_MBS.MBS_Ref_Taster_Ofenschieber_aussen, TIME_TSR, MBS_OperatingModes.F1)

    def AND_9(V_90000000000, V_90000000001):
        result = GVL.diNotAus_Kanal_A and GVL.diNotAus_Kanal_B
        return result
    V_90000000002 = AND_9(GVL.diNotAus_Kanal_A, GVL.diNotAus_Kanal_B)

    def MBS_MR01_AS_SecuringProcess(V_90000000002, V_90000000003, V_90000000004):
        result = MBS_OperatingModes.F1
        return result
    V_90000000005 = MBS_MR01_AS_SecuringProcess(V_90000000002, GVL_MBS.MBS_Ref_Taster_Ofenschieber_innen, MBS_OperatingModes.F1)

    def AND_10(V_100000000000, V_100000000001):
        result = GVL.diNotAus_Kanal_A and GVL.diNotAus_Kanal_B
        return result
    V_100000000002 = AND_10(GVL.diNotAus_Kanal_A, GVL.diNotAus_Kanal_B)

    def MBS_MR01_AS_HardeningProcess(V_100000000002, V_100000000003, V_100000000004):
        result = MBS_OperatingModes.F1
        return result
    V_100000000005 = MBS_MR01_AS_HardeningProcess(V_100000000002, GVL_MBS.MBS_Ref_Taster_Ofenschieber_innen, MBS_OperatingModes.F1)

    def AND_11(V_110000000000, V_110000000001):
        result = GVL.diNotAus_Kanal_A and GVL.diNotAus_Kanal_B
        return result
    V_110000000002 = AND_11(GVL.diNotAus_Kanal_A, GVL.diNotAus_Kanal_B)

    def MBS_VSG_AS_VerticalMove(V_110000000002, V_110000000003, V_110000000004, V_110000000005, V_110000000006):
        result = MBS_OperatingModes.F1
        return result
    V_110000000007 = MBS_VSG_AS_VerticalMove(V_110000000002, GVL_MBS.MBS_Ref_Taster_Sauger_Pos_DK, PowerOnDelay_Compressor, PowerOnDelay_Valve_VerticalMove, MBS_OperatingModes.F1)

    def AND_12(V_120000000000, V_120000000001):
        result = GVL.diNotAus_Kanal_A and GVL.diNotAus_Kanal_B
        return result
    V_120000000002 = AND_12(GVL.diNotAus_Kanal_A, GVL.diNotAus_Kanal_B)

    def MBS_VSG_AS_SuctionProcess_1(V_120000000002, V_120000000003, V_120000000004, V_120000000005, V_120000000006, V_120000000007):
        result = MBS_OperatingModes.F1
        return result
    V_120000000008 = MBS_VSG_AS_SuctionProcess_1(V_120000000002, GVL_MBS.MBS_Ref_Taster_Sauger_Pos_Ofen, GVL_MBS.MBS_Ref_Taster_Sauger_Pos_DK, PowerOnDelay_Compressor, PowerOnDelay_Valve_SuctionProcess, MBS_OperatingModes.F1)

    def AND_13(V_130000000000, V_130000000001):
        result = GVL.diNotAus_Kanal_A and GVL.diNotAus_Kanal_B
        return result
    V_130000000002 = AND_13(GVL.diNotAus_Kanal_A, GVL.diNotAus_Kanal_B)

    def MBS_VSG_AS_SuctionProcess_2(V_130000000002, V_130000000003, V_130000000004, V_130000000005, V_130000000006, V_130000000007):
        result = MBS_OperatingModes.F1
        return result
    V_130000000008 = MBS_VSG_AS_SuctionProcess_2(V_130000000002, GVL_MBS.MBS_Ref_Taster_Sauger_Pos_DK, GVL_MBS.MBS_Ref_Taster_Sauger_Pos_Ofen, PowerOnDelay_Compressor, PowerOnDelay_Valve_SuctionProcess, MBS_OperatingModes.F1)

    def AND_14(V_140000000000, V_140000000001):
        result = GVL.diNotAus_Kanal_A and GVL.diNotAus_Kanal_B
        return result
    V_140000000002 = AND_14(GVL.diNotAus_Kanal_A, GVL.diNotAus_Kanal_B)

    def MBS_VSG_AS_HorizontalMove_1(V_140000000002, V_140000000003, V_140000000004, V_140000000005, V_140000000006):
        result = MBS_OperatingModes.F1
        return result
    V_140000000007 = MBS_VSG_AS_HorizontalMove_1(V_140000000002, GVL_MBS.MBS_Ref_Taster_Sauger_Pos_Ofen, GVL_MBS.MBS_Ref_Taster_Sauger_Pos_DK, TIME_TSR, MBS_OperatingModes.F1)

    def AND_15(V_150000000000, V_150000000001):
        result = GVL.diNotAus_Kanal_A and GVL.diNotAus_Kanal_B
        return result
    V_150000000002 = AND_15(GVL.diNotAus_Kanal_A, GVL.diNotAus_Kanal_B)

    def MBS_VSG_AS_HorizontalMove_2(V_150000000002, V_150000000003, V_150000000004, V_150000000005, V_150000000006):
        result = MBS_OperatingModes.F1
        return result
    V_150000000007 = MBS_VSG_AS_HorizontalMove_2(V_150000000002, GVL_MBS.MBS_Ref_Taster_Sauger_Pos_DK, GVL_MBS.MBS_Ref_Taster_Sauger_Pos_Ofen, TIME_TSR, MBS_OperatingModes.F1)

    def AND_16(V_160000000000, V_160000000001):
        result = GVL.diNotAus_Kanal_A and GVL.diNotAus_Kanal_B
        return result
    V_160000000002 = AND_16(GVL.diNotAus_Kanal_A, GVL.diNotAus_Kanal_B)

    def MBS_DmPD_AS_RotationMove_1(V_160000000002, V_160000000003, V_160000000004, V_160000000005):
        result = MBS_OperatingModes.F1
        return result
    V_160000000006 = MBS_DmPD_AS_RotationMove_1(V_160000000002, GVL_MBS.MBS_Ref_Taster_DK_Pos_Sauger, GVL_MBS.MBS_Ref_Taster_DK_Pos_Saege, MBS_OperatingModes.F1)

    def AND_17(V_170000000000, V_170000000001):
        result = GVL.diNotAus_Kanal_A and GVL.diNotAus_Kanal_B
        return result
    V_170000000002 = AND_17(GVL.diNotAus_Kanal_A, GVL.diNotAus_Kanal_B)

    def MBS_DmPD_AS_RotationMove_2(V_170000000002, V_170000000003, V_170000000004, V_170000000005):
        result = MBS_OperatingModes.F1
        return result
    V_170000000006 = MBS_DmPD_AS_RotationMove_2(V_170000000002, GVL_MBS.MBS_Ref_Taster_DK_Pos_Sauger, GVL_MBS.MBS_Ref_Taster_DK_Pos_Foerderband, MBS_OperatingModes.F1)

    def AND_18(V_180000000000, V_180000000001):
        result = GVL.diNotAus_Kanal_A and GVL.diNotAus_Kanal_B
        return result
    V_180000000002 = AND_18(GVL.diNotAus_Kanal_A, GVL.diNotAus_Kanal_B)

    def MBS_DmPD_AS_RotationMove_3(V_180000000002, V_180000000003, V_180000000004, V_180000000005):
        result = MBS_OperatingModes.F1
        return result
    V_180000000006 = MBS_DmPD_AS_RotationMove_3(V_180000000002, GVL_MBS.MBS_Ref_Taster_DK_Pos_Saege, GVL_MBS.MBS_Ref_Taster_DK_Pos_Foerderband, MBS_OperatingModes.F1)

    def AND_19(V_190000000000, V_190000000001):
        result = GVL.diNotAus_Kanal_A and GVL.diNotAus_Kanal_B
        return result
    V_190000000002 = AND_19(GVL.diNotAus_Kanal_A, GVL.diNotAus_Kanal_B)

    def MBS_DmPD_AS_RotationMove_4(V_190000000002, V_190000000003, V_190000000004, V_190000000005):
        result = MBS_OperatingModes.F1
        return result
    V_190000000006 = MBS_DmPD_AS_RotationMove_4(V_190000000002, GVL_MBS.MBS_Ref_Taster_DK_Pos_Foerderband, GVL_MBS.MBS_Ref_Taster_DK_Pos_Saege, MBS_OperatingModes.F1)

    def AND_20(V_200000000000, V_200000000001):
        result = GVL.diNotAus_Kanal_A and GVL.diNotAus_Kanal_B
        return result
    V_200000000002 = AND_20(GVL.diNotAus_Kanal_A, GVL.diNotAus_Kanal_B)

    def MBS_DmPD_AS_RotationMove_5(V_200000000002, V_200000000003, V_200000000004, V_200000000005):
        result = MBS_OperatingModes.F1
        return result
    V_200000000006 = MBS_DmPD_AS_RotationMove_5(V_200000000002, GVL_MBS.MBS_Ref_Taster_DK_Pos_Foerderband, GVL_MBS.MBS_Ref_Taster_DK_Pos_Sauger, MBS_OperatingModes.F1)

    def AND_21(V_210000000000, V_210000000001):
        result = GVL.diNotAus_Kanal_A and GVL.diNotAus_Kanal_B
        return result
    V_210000000002 = AND_21(GVL.diNotAus_Kanal_A, GVL.diNotAus_Kanal_B)

    def MBS_DmPD_AS_RotationMove_6(V_210000000002, V_210000000003, V_210000000004, V_210000000005):
        result = MBS_OperatingModes.F1
        return result
    V_210000000006 = MBS_DmPD_AS_RotationMove_6(V_210000000002, GVL_MBS.MBS_Ref_Taster_DK_Pos_Saege, GVL_MBS.MBS_Ref_Taster_DK_Pos_Sauger, MBS_OperatingModes.F1)

    def AND_22(V_220000000000, V_220000000001):
        result = GVL.diNotAus_Kanal_A and GVL.diNotAus_Kanal_B
        return result
    V_220000000002 = AND_22(GVL.diNotAus_Kanal_A, GVL.diNotAus_Kanal_B)

    def MBS_DmPD_AS_HorizontalMove(V_220000000002, V_220000000003, V_220000000004, V_220000000005, V_220000000006):
        result = MBS_OperatingModes.F1
        return result
    V_220000000007 = MBS_DmPD_AS_HorizontalMove(V_220000000002, GVL_MBS.MBS_Ref_Taster_DK_Pos_Foerderband, PowerOnDelay_Compressor, PowerOnDelay_Valve_HorizontalMove, MBS_OperatingModes.F1)

    def AND_23(V_230000000000, V_230000000001):
        result = GVL.diNotAus_Kanal_A and GVL.diNotAus_Kanal_B
        return result
    V_230000000002 = AND_23(GVL.diNotAus_Kanal_A, GVL.diNotAus_Kanal_B)

    def MBS_MR02_AS_CuttingProcess(V_230000000002, V_230000000003, V_230000000004):
        result = MBS_OperatingModes.F1
        return result
    V_230000000005 = MBS_MR02_AS_CuttingProcess(V_230000000002, GVL_MBS.MBS_Ref_Taster_DK_Pos_Saege, MBS_OperatingModes.F1)

    def AND_24(V_240000000000, V_240000000001):
        result = GVL.diNotAus_Kanal_A and GVL.diNotAus_Kanal_B
        return result
    V_240000000002 = AND_24(GVL.diNotAus_Kanal_A, GVL.diNotAus_Kanal_B)

    def MBS_CB_AS_HorizontalMove(V_240000000002, V_240000000003, V_240000000004, V_240000000005, V_240000000006):
        result = MBS_OperatingModes.F1
        return result
    V_240000000007 = MBS_CB_AS_HorizontalMove(V_240000000002, GVL_MBS.MBS_Ref_Taster_DK_Pos_Foerderband, GVL_MBS.MBS_LS_Ende_Foerderband, PowerOffDelay_ConveyorBelt, MBS_OperatingModes.F1)
    MBS_ControlOfOutputs.A1 = V_10000000015
    print('Value of MBS_ControlOfOutputs.A1:', MBS_ControlOfOutputs.A1)
    GVL_MBS.MBS_MOT_Foerderband_vorwaerts = V_20000000036
    print('Value of GVL_MBS.MBS_MOT_Foerderband_vorwaerts:', GVL_MBS.MBS_MOT_Foerderband_vorwaerts)
    Init_MR01_HorizontalForwards = V_30000000008
    print('Value of Init_MR01_HorizontalForwards:', Init_MR01_HorizontalForwards)
    Init_VSG_HorizontalForwards = V_40000000008
    print('Value of Init_VSG_HorizontalForwards:', Init_VSG_HorizontalForwards)
    Init_DmPD_RotateCW = V_50000000009
    print('Value of Init_DmPD_RotateCW:', Init_DmPD_RotateCW)
    Init_MR02_CuttingProcess = V_60000000007
    print('Value of Init_MR02_CuttingProcess:', Init_MR02_CuttingProcess)
    MR01_MoveOvenslideIn = V_70000000007
    print('Value of MR01_MoveOvenslideIn:', MR01_MoveOvenslideIn)
    MR01_MoveOvenslideOut = V_80000000007
    print('Value of MR01_MoveOvenslideOut:', MR01_MoveOvenslideOut)
    MR01_Compressor_SecuringProcess = V_90000000005
    print('Value of MR01_Compressor_SecuringProcess:', MR01_Compressor_SecuringProcess)
    MR01_HardeningProcess = V_100000000005
    print('Value of MR01_HardeningProcess:', MR01_HardeningProcess)
    VSG_Compressor_VerticalMove = V_110000000007
    print('Value of VSG_Compressor_VerticalMove:', VSG_Compressor_VerticalMove)
    VSG_Compressor_SuctionProcessToDmPD = V_120000000008
    print('Value of VSG_Compressor_SuctionProcessToDmPD:', VSG_Compressor_SuctionProcessToDmPD)
    VSG_Compressor_SuctionProcessToMR01 = V_130000000008
    print('Value of VSG_Compressor_SuctionProcessToMR01:', VSG_Compressor_SuctionProcessToMR01)
    VSG_MoveHorizontalForwards = V_140000000007
    print('Value of VSG_MoveHorizontalForwards:', VSG_MoveHorizontalForwards)
    VSG_MoveHorizontalBackwards = V_150000000007
    print('Value of VSG_MoveHorizontalBackwards:', VSG_MoveHorizontalBackwards)
    DmPD_RotateCWformVSGtoMR02 = V_160000000006
    print('Value of DmPD_RotateCWformVSGtoMR02:', DmPD_RotateCWformVSGtoMR02)
    DmPD_RotateCWromVSGtoCB = V_170000000006
    print('Value of DmPD_RotateCWromVSGtoCB:', DmPD_RotateCWromVSGtoCB)
    DmPD_RotateCWfromMR02toCB = V_180000000006
    print('Value of DmPD_RotateCWfromMR02toCB:', DmPD_RotateCWfromMR02toCB)
    DmPD_RotateCCWformCBtoMR02 = V_190000000006
    print('Value of DmPD_RotateCCWformCBtoMR02:', DmPD_RotateCCWformCBtoMR02)
    DmPD_RotateCCWformCBtoVSG = V_200000000006
    print('Value of DmPD_RotateCCWformCBtoVSG:', DmPD_RotateCCWformCBtoVSG)
    DmPD_RotateCCWformMR02toVSG = V_210000000006
    print('Value of DmPD_RotateCCWformMR02toVSG:', DmPD_RotateCCWformMR02toVSG)
    DmPD_Compressor_HorizontalMove = V_220000000007
    print('Value of DmPD_Compressor_HorizontalMove:', DmPD_Compressor_HorizontalMove)
    MR02_CuttingProcess = V_230000000005
    print('Value of MR02_CuttingProcess:', MR02_CuttingProcess)
    CB_MoveForwards = V_240000000007
    print('Value of CB_MoveForwards:', CB_MoveForwards)
    return {'MBS_ControlOfOutputs.A1': MBS_ControlOfOutputs.A1, 'GVL_MBS.MBS_MOT_Foerderband_vorwaerts': GVL_MBS.MBS_MOT_Foerderband_vorwaerts, 'Init_MR01_HorizontalForwards': Init_MR01_HorizontalForwards, 'Init_VSG_HorizontalForwards': Init_VSG_HorizontalForwards, 'Init_DmPD_RotateCW': Init_DmPD_RotateCW, 'Init_MR02_CuttingProcess': Init_MR02_CuttingProcess, 'MR01_MoveOvenslideIn': MR01_MoveOvenslideIn, 'MR01_MoveOvenslideOut': MR01_MoveOvenslideOut, 'MR01_Compressor_SecuringProcess': MR01_Compressor_SecuringProcess, 'MR01_HardeningProcess': MR01_HardeningProcess, 'VSG_Compressor_VerticalMove': VSG_Compressor_VerticalMove, 'VSG_Compressor_SuctionProcessToDmPD': VSG_Compressor_SuctionProcessToDmPD, 'VSG_Compressor_SuctionProcessToMR01': VSG_Compressor_SuctionProcessToMR01, 'VSG_MoveHorizontalForwards': VSG_MoveHorizontalForwards, 'VSG_MoveHorizontalBackwards': VSG_MoveHorizontalBackwards, 'DmPD_RotateCWformVSGtoMR02': DmPD_RotateCWformVSGtoMR02, 'DmPD_RotateCWromVSGtoCB': DmPD_RotateCWromVSGtoCB, 'DmPD_RotateCWfromMR02toCB': DmPD_RotateCWfromMR02toCB, 'DmPD_RotateCCWformCBtoMR02': DmPD_RotateCCWformCBtoMR02, 'DmPD_RotateCCWformCBtoVSG': DmPD_RotateCCWformCBtoVSG, 'DmPD_RotateCCWformMR02toVSG': DmPD_RotateCCWformMR02toVSG, 'DmPD_Compressor_HorizontalMove': DmPD_Compressor_HorizontalMove, 'MR02_CuttingProcess': MR02_CuttingProcess, 'CB_MoveForwards': CB_MoveForwards}